# Transkribus REST API Client

## Basics I: Talking to a very simple API
Playing around with a free and simple API is the best way to learn how to use the `requests` package. The Open Notify API for example provides information about the International Space Station. 

Where to get help: 
* documentation of this API: http://api.open-notify.org/
* the `requests` package: https://docs.python-requests.org/en/master/
* the `json` package: https://docs.python.org/3/library/json.html

In [1]:
""" List the astronauts that are currently on the ISS
    using the Open Notify API: http://api.open-notify.org/ """

import requests
import json

# Build the URL using the "astros.json" endpoint:
api_base_url = "http://api.open-notify.org/"
endpoint = "astros.json"
url = api_base_url + endpoint

# Make the request and store the response:
response = requests.get(url)

# Evaluate the response:
if response:
    try:
        json_response = response.json()
        print("The raw JSON response sent by the server:")
        print(json.dumps(json_response))   
        print("\nPeople on the ISS:")
        for astronaut in json_response['people']:
            if astronaut['craft'] == "ISS":
                print("– " + astronaut['name'])
    except:
        print(f"ERROR: Something went wrong. {response.content}")
else:
    print(f"ERROR: Something went wrong. HTTP status code: {response.status_code}")

The raw JSON response sent by the server:
{"people": [{"name": "Mark Vande Hei", "craft": "ISS"}, {"name": "Oleg Novitskiy", "craft": "ISS"}, {"name": "Pyotr Dubrov", "craft": "ISS"}, {"name": "Thomas Pesquet", "craft": "ISS"}, {"name": "Megan McArthur", "craft": "ISS"}, {"name": "Shane Kimbrough", "craft": "ISS"}, {"name": "Akihiko Hoshide", "craft": "ISS"}, {"name": "Nie Haisheng", "craft": "Tiangong"}, {"name": "Liu Boming", "craft": "Tiangong"}, {"name": "Tang Hongbo", "craft": "Tiangong"}], "number": 10, "message": "success"}

People on the ISS:
– Mark Vande Hei
– Oleg Novitskiy
– Pyotr Dubrov
– Thomas Pesquet
– Megan McArthur
– Shane Kimbrough
– Akihiko Hoshide


## Basics II: Talking to the Transkribus REST-API
Using the same approach as above, we can access the Transkribus REST-API. First we have to log in, then we can access the endpoints, finally we have to log out.

Where to get help: 
* the documentation of the Transkribus REST-API: https://readcoop.eu/transkribus/docu/rest-api/ 
* a list of all available endpoints: https://transkribus.eu/TrpServer/Swadl/wadl.html
* the `objectify` API of the `lxml` package: https://lxml.de/objectify.html

In [2]:
import requests
import sys
from lxml import objectify

# Define building blocks for the endpoints:
api_base_url = "https://transkribus.eu/TrpServer/rest/"
login_endpoint = "auth/login"
collections_endpoint = "collections/list"
logout_endpoint = "auth/logout"

# Log in:
## Prepare the payload for the POST request:
credentials = {'user': 'YOUR_USER_NAME',
               'pw': 'YOUR_PASSWORD'}
## Create the POST request (the requests package converts the credentials into JSON automatically):
response = requests.post(api_base_url + login_endpoint, data=credentials)

## Evaluate the response:
if response:
    r = objectify.fromstring(response.content)
    print(f"TRANSKRIBUS: User {r.firstname} {r.lastname} ({r.userId}) logged in successfully.")
    session_id = str(r.sessionId)
else:
    sys.exit("TRANSKRIBUS: Login failed.")

# Get the list of collections (using a GET request):
cookies = dict(JSESSIONID=session_id)
response = requests.get(api_base_url + collections_endpoint, cookies=cookies)
if response:
    collections = response.json()
    print("TRANSKRIBUS: Your collections:")
    for collection in collections:
        print(f"– {collection['colName']}, ({collection['nrOfDocuments']} documents)")
else:
    sys.exit("TRANSKRIBUS: Could not retrieve collections.")

# Log out (using a POST request):
response = requests.post(api_base_url + logout_endpoint, cookies=cookies)
if response:
    print("TRANSKRIBUS: Logged out successfully!")
else:
    sys.exit("TRANSKRIBUS: Logout failed.")


TRANSKRIBUS: User Markus Müller (XXXXX) logged in successfully.
TRANSKRIBUS: Your collections:
– XYZ@example.de Collection, (14 documents)
– FerusAlpha, (24 documents)
– FerusBeta, (8 documents)
– FerusArchiv, (12 documents)
– Transkribus Spellchecker Demo, (1 documents)
TRANSKRIBUS: Logged out successfully!


# A custom Transkribus REST-API Client class
You may have noticed that the code above is very repetetive: For example, in every step we use the same `response = requests.get` command. Therefore, it is more convenient to offload the repetetive code into reusable functions and wrap these functions in a class.

In [3]:
import requests
import json
from lxml import objectify, etree
from pprint import pprint

In [4]:
class Transkribus_Web():
    """ The Transkribus_Web class implements the communication with the 
        Transkribus REST API. 
        
        All available endpoints: https://transkribus.eu/TrpServer/Swadl/wadl.html
        Documentation: https://readcoop.eu/transkribus/docu/rest-api/
        Official TranskribusPyClient: https://github.com/Transkribus/TranskribusPyClient """
    
    def __init__(self, api_base_url="https://transkribus.eu/TrpServer/rest/"):
        self.cols = {}
        self.api_base_url = api_base_url
        self.session_id = False
        self.cache = False
    
    # Internal helper functions:
    
    def _url(self, endpoint):
        """ Helper function that returns a full URL for requests to the REST API, 
            i.e. the API base URL + the relative path of the endpoint. """
        
        return self.api_base_url + endpoint
    
    # Core functionality: login, logout, send GET requests
    
    def login(self, username, password):
        """ Performs a login and stores the SESSIONID in the "session_id" variable
            of this class. """
        
        credentials = {'user': username,
                       'pw': password}
        response = requests.post(self._url("auth/login"), data=credentials)
        if response:
            r = objectify.fromstring(response.content)
            print(f"TRANSKRIBUS: User {r.firstname} {r.lastname} ({r.userId}) logged in successfully.")
            self.session_id = str(r.sessionId)
            return str(r.sessionId)
        else:
            print("TRANSKRIBUS: Login failed. HTTP status:", response.status_code)
            return False
    
    def logout(self):
        """ Logs out and sets the "session_id" variable to False. """
        
        cookies = dict(JSESSIONID=self.session_id)
        response = requests.post(self._url("auth/logout"), cookies=cookies)
        if response:
            self.session_id = False
            print("TRANSKRIBUS: Logged out successfully.")
            return True
        else:
            print("TRANSKRIBUS: Logout failed. HTTP status:", response.status_code, response.content)
            return False

    def verify(self, username, password):
        """ Logs in and logs out to check whether the credentials
            are valid on the Transkribus server. 
            Returns True or False. """
        
        session_id = self.login(username, password)
        if session_id:
            self.logout(session_id)
            return True
        else: 
            return False        
        
    def request_endpoint(self, endpoint):
        """ Sends a GET request to a Transkribus API endpoint, the 
            "endpoint" argument being a relative path to the REST API endpoint

            Cf. the list of available endpoints:
            https://transkribus.eu/TrpServer/Swadl/wadl.html

            Depending on the content type (JSON or XML), 
            the function tries to decode the raw content of the response.
            It returns a json object or an "objectify" object (lxml). If
            the conversion fails the raw content is returned. """

        cookies = dict(JSESSIONID=self.session_id)
        
        response = requests.get(self._url(endpoint), cookies=cookies)

        if response:
            try:
                json = response.json()
                return json
            except:
                try:
                    xml = objectify.fromstring(response.content)
                    return xml
                except:
                    return response.content  # fallback option if the server returns just text
        else:
            print(f'TRANSKRIBUS: ERROR when requesting "{endpoint}". HTTP status:', response.status_code)
            return False
    
    # Convenience functions to query certain endpoints: 
    
    def get_collections(self):
        """ Get the metadata of the owner's collections. 
            Returns a dict if successful or False if not. """
        
        endpoint = "collections/list"
        collections = self.request_endpoint(endpoint)
        return collections if collections else False
    
    def get_documents_in_collection(self, colId):
        """ Get the metadata of a collection. 
            Returns a dict if successful or False if not.
            
            colId -- collection ID in Transkribus (int) """
        
        endpoint = f"collections/{colId}/list"
        documents = self.request_endpoint(endpoint)
        return documents if documents else False

    def get_pages_in_document(self, colId, docId):
        """ Get the basic metadata of the pages in a document. 
            Returns a dict if successful or False if not.
           
            colId -- collection ID in Transkribus (int) 
            docId -- document ID in Transkribus (int) 
            pageNr -- page ID in Transkribus (int) """
        
        endpoint = f"collections/{colId}/{docId}/pages"
        pages = self.request_endpoint(endpoint)
        return pages if pages else False
        
    def get_page_xml(self, colId, docId, pageNr):
        """ Get the XML content of a page. 
            Returns an "objectify" object (lxml) or False if not successful. 
            
            colId -- collection ID in Transkribus (int) 
            docId -- document ID in Transkribus (int) 
            pageNr -- page ID in Transkribus (int) 
            
            The returned object X has two attributes: X.Metadata and X.Page. 
            X.Page is empty if there are no transcripts yet. 
            If there exists a transcription X.Page has further attributes
            (i and j are list indices counting from 0):
            
            X.Page.Metadata
                  .ReadingOrder
                  .values()   -> list containing imgFileName, width (px), height (px)
                  .TextRegion[i].Coords.attrib['points']               -> coordinates of the whole text region (1)
                                .TextEquiv.Unicode                     -> utf-8 string of the transcription of the whole text region
                                .TextLine[j].Coords.attrib['points']   -> coordinates of this line (1) (2)
                                            .BaseLine.attrib['points'] -> coordinates of this baseline (2)
                                            .TextEquiv.Unicode         -> utf-8 string of the transcription
            
            (1) Instead of .attrib['points'] you can say .values()[0].
            (2) The line is a polygon around the line of text, the BaseLine is a line below the text.
            
            You can check for the existence of attributes: if hasattr(X.Page, "TextRegion")…
            Get a list of existing attributes: X.Page.__dict__

            """
        
        endpoint = f"collections/{colId}/{docId}/{pageNr}/text"
        page_xml = self.request_endpoint(endpoint)
        return page_xml if page_xml is not None else False
        
    def upload_page_xml(self, colId, docId, pageNr, new_status, page_xml):
        """ Upload page XML data to the Transkribus server using a POST request. 
            Returns True or False.
        
            colId -- collection ID in Transkribus (int) 
            docId -- document ID in Transkribus (int) 
            pageNr -- page ID in Transkribus (int) 
            new_status -- new_status of the page. Possible values are NEW, IN_PROGRESS, DONE, FINAL, GT. 
            
            tsId = transcript ID of the last version of this transcription (int).
            After the upload Transkribus will generate a new transcription Id and save 
            the old one as the 'parentTsId' of the new transcription. """

        # Get the transcript ID of the latest transcription:
        current_transcript = self.request_endpoint(f"collections/{colId}/{docId}/{pageNr}/curr")
        if current_transcript:
            tsId = current_transcript['tsId']
        else:
            return False
        
        headers = {'Content-Type': 'text/xml'} 
        cookies = dict(JSESSIONID=self.session_id)
        params = {'status': new_status,
                  'parent': tsId,
                  'overwrite': 'false'}
        # convert the page_xml object to a pretty utf-8 string:
        data = etree.tostring(page_xml, pretty_print=True, xml_declaration=True).decode("utf-8")

        response = requests.post(self._url(f"collections/{colId}/{docId}/{pageNr}/text"), 
                                 headers=headers,
                                 params=params,
                                 cookies=cookies, 
                                 data=data)

        if response:
            print(f"Uploaded page {colId}/{docId}/{pageNr} successfully: {response.status_code}")
            print(response.content.decode(encoding="utf-8"))
            return True
        else:
            print(f"ERROR while uploading {colId}/{docId}/{pageNr}: {response.status_code}")
            print(response.content.decode(encoding="utf-8"))
            return False

    def update_page_status(self, colId, docId, pageNr, new_status):
        """ Update the status of the transcript with the transcript ID tsId with new_status.
            Returns True or False. 
            
            colId -- collection ID in Transkribus (int) 
            docId -- document ID in Transkribus (int) 
            pageNr -- page ID in Transkribus (int) 
            new_status -- new_status of the page. Possible values are NEW, IN_PROGRESS, DONE, FINAL, GT. """

        # Get the transcript ID of the latest transcription:
        current_transcript = self.request_endpoint(f"collections/{colId}/{docId}/{pageNr}/curr")
        if current_transcript:
            tsId = current_transcript['tsId']
        else:
            return False
        
        cookies = dict(JSESSIONID=self.session_id)
        params = {'status': new_status}
        
        endpoint = f"collections/{colId}/{docId}/{pageNr}/{tsId}"
        response = requests.post(self._url(endpoint),
                                 params=params,
                                 cookies=cookies)
        
        if response:
            print(f"TRANSKRIBUS: Updated status of page {pageNr} to {new_status} in collection {colId}, document {docId}.")
            print(response.content.decode(encoding="utf-8"))
            return True
        else:
            print(f"TRANSKRIBUS: ERROR: Could not update status of page {pageNr} to {new_status} in collection {colId}, document {docId}.")
            print(response.content.decode(encoding="utf-8"))
            return False
            
    
        

In [5]:
client = Transkribus_Web()
session_id = client.login("YOUR_USER_NAME", "YOUR_PASSWORD")

TRANSKRIBUS: User Markus Müller (XXXXX) logged in successfully.


In [6]:
# Get a list of all your collections:
client.get_collections()

# You could use the "request_endpoint" function to achieve the same result:
#collections = client.request_endpoint("collections/list")

[{'type': 'trpCollection',
  'colId': 19831,
  'colName': 'XYZ@example.de Collection',
  'description': 'XYZ@example.de',
  'crowdsourcing': False,
  'elearning': False,
  'pageId': 2950070,
  'url': 'https://files.transkribus.eu/Get?id=ZGQUVERWOEJLMXKWFKGOAQMO&fileType=view',
  'thumbUrl': 'https://files.transkribus.eu/Get?id=ZGQUVERWOEJLMXKWFKGOAQMO&fileType=thumb',
  'nrOfDocuments': 14,
  'role': 'Owner',
  'accountingStatus': 1},
 {'type': 'trpCollection',
  'colId': 37299,
  'colName': 'FerusAlpha',
  'description': "XYZ@example.de. Transcription of John Wild's commentary on the Gospel of John.",
  'crowdsourcing': False,
  'elearning': False,
  'pageId': 5263161,
  'url': 'https://files.transkribus.eu/Get?id=IXVIHRDIUIIEPNMGCNEBPVEY&fileType=view',
  'thumbUrl': 'https://files.transkribus.eu/Get?id=IXVIHRDIUIIEPNMGCNEBPVEY&fileType=thumb',
  'nrOfDocuments': 24,
  'role': 'Owner',
  'accountingStatus': 1},
 {'type': 'trpCollection',
  'colId': 47534,
  'colName': 'FerusBeta',
  

In [7]:
# Check the output of get_collections() above,
# pick one of the collections and store its colId here:
colId = XXXXX

# Get the metadata of all the documents in this collection:
client.get_documents_in_collection(colId)

[{'type': 'trpDocMetadata',
  'docId': 202832,
  'title': 'Wild1550-Joh1,VD16W2963,Teil_2',
  'uploadTimestamp': 1567182500354,
  'uploader': 'XYZ@example.de',
  'uploaderId': XXXXX,
  'nrOfPages': 34,
  'pageId': 8477570,
  'url': 'https://files.transkribus.eu/Get?id=QXLZCOEWXQGNZECIZYVSMSDN&fileType=view',
  'thumbUrl': 'https://files.transkribus.eu/Get?id=QXLZCOEWXQGNZECIZYVSMSDN&fileType=thumb',
  'status': 0,
  'fimgStoreColl': 'TrpDoc_DEA_202832',
  'origDocId': 0,
  'collectionList': {'colList': [{'colId': 47534,
     'colName': 'FerusBeta',
     'description': 'created by XYZ@example.de',
     'crowdsourcing': False,
     'elearning': False,
     'nrOfDocuments': 0},
    {'colId': 59163,
     'colName': 'FerusArchiv',
     'description': 'created by XYZ@example.de',
     'crowdsourcing': False,
     'elearning': False,
     'nrOfDocuments': 0}]}},
 {'type': 'trpDocMetadata',
  'docId': 306607,
  'title': 'Wild1550-Joh,VD16W2963,new',
  'author': 'Johann Wild (Ferus, 1495-1554)'

In [8]:
# Pick a document and store its docId here:
docId = XXXXXX

# Get the metadata of all the pages in this document:
client.get_pages_in_document(colId, docId)

# The same as:
#client.request_endpoint(f"collections/{colId}/{docId}/pages")

[{'pageId': 12144537,
  'docId': 306608,
  'pageNr': 1,
  'key': 'LYGZFELUMETPKPJMIBTUPKMS',
  'imageId': 8478757,
  'url': 'https://files.transkribus.eu/Get?id=LYGZFELUMETPKPJMIBTUPKMS&fileType=view',
  'thumbUrl': 'https://files.transkribus.eu/Get?id=LYGZFELUMETPKPJMIBTUPKMS&fileType=thumb',
  'imgFileName': 'Wild1559o-Mt,Mainz,BW,VD16W2965,__0011.png',
  'tsList': {'transcripts': [{'tsId': 33934402,
     'parentTsId': 27409630,
     'key': 'PNGFVQCMUUKHJPSFCVEHGQJZ',
     'pageId': 12144537,
     'docId': 306608,
     'pageNr': 1,
     'url': 'https://files.transkribus.eu/Get?id=PNGFVQCMUUKHJPSFCVEHGQJZ',
     'status': 'FINAL',
     'userName': 'XYZ@example.de',
     'userId': XXXXX,
     'timestamp': 1594302136870,
     'md5Sum': '',
     'nrOfRegions': 3,
     'nrOfTranscribedRegions': 3,
     'nrOfWordsInRegions': 390,
     'nrOfLines': 57,
     'nrOfTranscribedLines': 57,
     'nrOfWordsInLines': 444,
     'nrOfWords': 0,
     'nrOfTranscribedWords': 0}]},
  'width': 2085,
  'h

In [9]:
# Pick a page and store its pageNr here:
pageNr = XX

# Let's check the Id of the last transcript:
client.request_endpoint(f"collections/{colId}/{docId}/{pageNr}/curr")['tsId']

33934412

In [10]:
# Get the page_xml data of this page:
doc = client.get_page_xml(colId, docId, pageNr)

# Same as:
#doc = client.request_endpoint(f"collections/{colId}/{docId}/{pageNr}/text")

In [151]:
# Let's play around with the page_xml object:
# (Comment out and run one of the following lines. You may have to modify the indices of the dicts)

#doc.Page.TextRegion[1].TextLine[1].Coords.values()
#doc.Page.TextRegion[1].TextEquiv.Unicode
#dir(doc.Metadata)
#print(etree.tostring(doc, pretty_print=True, xml_declaration=True).decode("utf-8"))

<?xml version='1.0' encoding='ASCII'?>
<PcGts xmlns="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15 http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15/pagecontent.xsd">
  <Metadata>
    <Creator>prov=University of Rostock/Institute of Mathematics/CITlab/Gundram Leifert/gundram.leifert@uni-rostock.de:name=Printed-Wild1550-In_Ioannem-VD16W2963(htr_id=16391)::::v=2.4.2
prov=University of Rostock/Institute of Mathematics/CITlab/Tobias Gruening/tobias.gruening@uni-rostock.de:name=de.uros.citlab.module.baseline2polygon.B2PSeamMultiOriented:v=2.4.2
prov=University of Rostock/Institute of Mathematics/CITlab/Tobias Gruening/tobias.gruening@uni-rostock.de:name=/net_tf/LA73_249_0mod360.pb:de.uros.citlab.segmentation.CITlab_LA_ML:v=2.4.2
TRP</Creator>
    <Created>2019-08-30T18:27:41.219+02:00</Created>
    <LastChange>2021-05-26T2

## Additional functions
Having tested the `Transkribus_Web` class, we should build some additional functions that help us to deal with the data downloaded from Transkribus. The following code makes use of the [XPath](https://www.w3schools.com/xml/xml_xpath.asp) standard of XML. (The `find` method of the `objectify` API of `lxml` can understand XPath.)

In [5]:
def check_page(page_xml):
    """ Make sure that the page_xml contains
        – TextRegions
        – Baselines
        – TextEquiv, i.e. actual text in the lines. 
        This is useful for further processing of the data to prevent crashes.
        
        page_xml -- a lxml.objectify object of a page in Transkribus """
        
    ns = "{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}"
    
    if page_xml.find(f".//{ns}TextRegion") is None:
        return "PAGE-XML: ERROR: No TextRegions found."
    if page_xml.find(f".//{ns}Baseline") is None:
        return "PAGE-XML: ERROR: No BaseLines found."
    if page_xml.find(f".//{ns}TextEquiv") is None:
        return "PAGE-XML: ERROR: Lines contain no text."

    return True

check_page(doc)

True

The `page_xml` contains a lot of data and metadata. For us, the most interesting pieces are the lines containing the actual transcription. Since every line has a line number and is part of a TextRegion which also has a number, it is convenient to write a function that extracts these attributes:

In [6]:
import re

def get_custom_attributes(string):
    custom_attributes = re.compile(r'\{(\w*?):(\w*?);\}')
    return dict(custom_attributes.findall(string))

# Store the namespace string used by the Transkribus page_xml format:
ns = "{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}"

for line in doc.Page.iter(f"{ns}TextLine"): # Cf. the section "tree iteration" in https://lxml.de/tutorial.html
    # Get the attributes of the TextRegion:
    custom_attributes = get_custom_attributes(line.getparent().attrib['custom'])
    # In the following line you could filter TextRegions tagged with a specific tag (like "paragraph"):
    if custom_attributes.get("type"): # No filtering at the moment.
        print(line.attrib['id'], line.TextEquiv.Unicode)

r2l1 TIONE
r2l2 NoO.
r2l3 AN
r2l4 s, qui onere leg
r2l5 , à quo mitiora ſperat
r2l6 hoc eſt, alium quempia
r2l7 Demum ad Ioan-
r2l8 nos liberet, & carnaliter ſecuros faciat, &c̄.
r2l9 ore Ioannis (nam & ipſum uidebant contra-
r2l10 nem mittunt, non a
r2l11 rium ſuis affectibus) ſed odio & inuidia Chriſti, ut ſi hunc hono-
r2l12 rem Ioannes agnoſceret, haberent iuſtam occaſionem in Chriſtur
r2l13 Ioan. 5. ſęuiendi, & doctrinam eius damnandi. Hinc ipſe Chriſtus dicit:
r2l14 Vos miſiſtis ad Ioannem, & uoluiſtis ad tempus exultare in luce
r2l15 iſtis eo interin
r2l16 eius, id eſt, obtuliſtis ei honorem Meſsiæ, & uo
r2l17 Huc pertinet, quod Chri=
r2l18 primeretis me, &c̄.
r2l19 abuti, donec o
r2l20 ſtus de ſeipſo ſub parabola nobilis cuiuſdam, qui peregre profectus,
r2l21 t, eius ode-
r2l22 s, inqʒ
r2l23 n quoddam ſibi ſubijcere uolebat. 
r2l24 Luc. 19. Regni
r2l25 s hunc re-
r2l26 rant eum, & legationem miſerunt, dicentes: Nolum
r2l27 ere mundi inge
r2l28 Ex his igitur ſatis
r2l29 gnare ſu

Oh my God, there are so many transcription errors on this page!! The following snippet shows how to manipulate a line in the transcription:

In [11]:
# print the first line:
print(doc.Page.TextRegion[1].TextLine[0].TextEquiv.Unicode)
# modify the first line:
doc.Page.TextRegion[1].TextLine[0].TextEquiv.Unicode = "TIONE"
# print the first line:
print(doc.Page.TextRegion[1].TextLine[0].TextEquiv.Unicode)

tet. Ad vltimum, quem Chriſtum audire piget, is nec me aliud loquentem au-
TIONE


If you upload the manipulated transcription, the changes you made in the previous step will be stored on the Transkribus server:

In [81]:
client.upload_page_xml(colId, docId, pageNr, "GT", doc)

# "GT" means "Ground Truth" and is the status of the page. 
# Possible statuses are: "NEW", "IN_PROGRESS", "DONE", "FINAL", "GT"

Uploaded page 47534/202832/2 successfully: 200
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><trpTranscriptMetadata><tsId>58367253</tsId><parentTsId>58367242</parentTsId><key>WWWVWDDMJHDPDMHWDRAAHNKM</key><pageId>8477571</pageId><docId>202832</docId><pageNr>2</pageNr><url>https://files.transkribus.eu/Get?id=WWWVWDDMJHDPDMHWDRAAHNKM</url><status>GT</status><userName>XYZ@example.de</userName><userId>XXXXX</userId><timestamp>1622058344411</timestamp><md5Sum></md5Sum><nrOfRegions>2</nrOfRegions><nrOfTranscribedRegions>1</nrOfTranscribedRegions><nrOfWordsInRegions>352</nrOfWordsInRegions><nrOfLines>70</nrOfLines><nrOfTranscribedLines>70</nrOfTranscribedLines><nrOfWordsInLines>420</nrOfWordsInLines><nrOfWords>0</nrOfWords><nrOfTranscribedWords>0</nrOfTranscribedWords></trpTranscriptMetadata>


True

Another frequent operaion is to update the status of a page:

In [96]:
# Change the status of the page (but nothing else):
client.update_page_status(47534, 202832, 2, "GT")

TRANSKRIBUS: Updated status of page 2 to GT in collection 47534, document 202832.



True

## Log Out
Last but not least, we should always log out in the end…

In [12]:
# … and finally, log out:
client.logout()

TRANSKRIBUS: Logged out successfully.


True